In [1]:
import torch
print(torch.__version__)

2.6.0+cu124


In [4]:
#when we write requires_grad= True, it creates computation graph. can write the argument only with floating numbers
x = torch.tensor(3.0,requires_grad=True)

print(x)


y = x ** 2

print(y)

#It calculate derivative (dy/dx)
y.backward()

print(x.grad)

tensor(3., requires_grad=True)
tensor(9., grad_fn=<PowBackward0>)
tensor(6.)


In [5]:
x = torch.tensor(2.0,requires_grad=True)
print(x)

y = x ** 3
print(y)

z = torch.sin(y)
print(z)

#It calculate derivative (dz/dx)
z.backward()

print(x.grad)

tensor(2., requires_grad=True)
tensor(8., grad_fn=<PowBackward0>)
tensor(0.9894, grad_fn=<SinBackward0>)
tensor(-1.7460)


**Autograd in Neural Network**

In [6]:
x = torch.tensor(6.7) #Input feature
y = torch.tensor(0.0) #True label(binary)

w = torch.tensor(1.0) #Weight
b = torch.tensor(0.0) #Bias


#Binary cross entropy loss
def binary_cross_entropy_loss(prediction,target):
  epsilon = 1e-8
  prediction = torch.clamp(prediction,epsilon,1-epsilon)
  loss = -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))
  return loss


#Forward pass
z = w * x + b #weighted sum
y_pred = torch.sigmoid(z)

#Compute binary cross entropy loss
loss = binary_cross_entropy_loss(y_pred,y)

print(loss)



tensor(6.7012)


In [8]:
#Derivatives:

#1. dL/d(y_pred): Loss wrt prediction(y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred * (1-y_pred))

#2.dy_pred/dz = Prediction(y_pred) wrt to z(sigmoid derivative)
dy_pred_dz = y_pred * (1-y_pred)

#3.dz/dw  and dz/db: z wrt to w and b
dz_dw = x
dz_db = 1

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

print(f"Manual gradient of loss wrt to weight is {dL_dw}")
print(f"Manual gradient of loss wrt to bias is {dL_db}")



Manual gradient of loss wrt to weight is 6.691762447357178
Manual gradient of loss wrt to bias is 0.998770534992218


**Implementing with Autograd**

In [10]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)
print(x)
print(y)

w = torch.tensor(1.0,requires_grad=True)
b = torch.tensor(0.0,requires_grad=True)
print(w)
print(b)


tensor(6.7000)
tensor(0.)
tensor(1., requires_grad=True)
tensor(0., requires_grad=True)


In [11]:
z = w * x + b
print(z)

tensor(6.7000, grad_fn=<AddBackward0>)


In [12]:
y_pred = torch.sigmoid(z)
print(y_pred)

tensor(0.9988, grad_fn=<SigmoidBackward0>)


In [13]:
loss = binary_cross_entropy_loss(y_pred,y)
print(loss)

tensor(6.7012, grad_fn=<NegBackward0>)


In [14]:
loss.backward()

In [16]:
w_grad = w.grad
b_grad = b.grad

print(f"Auto grad Gradient of loss wrt to weight is {w_grad}")
print(f"Auto grad Gradient of loss wrt to bias is {b_grad}")

Auto grad Gradient of loss wrt to weight is 6.6917619705200195
Auto grad Gradient of loss wrt to bias is 0.9987704753875732


**Implementing Autograd for multidimensional input**

In [17]:
x = torch.tensor([1.0,2.0,3.0],requires_grad=True)
print(x)

y = (x ** 2).mean()
print(y)

y.backward()
print(x.grad)

tensor([1., 2., 3.], requires_grad=True)
tensor(4.6667, grad_fn=<MeanBackward0>)
tensor([0.6667, 1.3333, 2.0000])


**Clearing gradient(necessary for multi forward and backward pass)**

In [25]:
x = torch.tensor(2.0,requires_grad=True)
print(x)

tensor(2., requires_grad=True)


In [42]:
y = (x ** 2)
print(y)

tensor(4., grad_fn=<PowBackward0>)


In [43]:
y.backward()

In [44]:
print(x.grad)

tensor(8.)


In [45]:
#Try running above code cell starting from y and run without running this cell multiple times to observe the difference in x.grad value
x.grad.zero_()

tensor(0.)

In [46]:
#Disable gradient tracking(useful for forward pass only[inference time])

#Option 1: requires_grad_(False)
#Option 2: detach()
#Option 3: torch.no_grad()

x.requires_grad_(False)
print(x)

y = x ** 2
print(y)

#y.backward() - will throw error


tensor(2.)
tensor(4.)


In [49]:
#Option 2

x = torch.tensor(2.0,requires_grad=True)
print(x)

z = x.detach()
print(z)

y = x ** 2
print(y)

y1 = z ** 2
print(y1)

y.backward()

y1.backward() #It will throw error since gradient is disabled

tensor(2., requires_grad=True)
tensor(2.)
tensor(4., grad_fn=<PowBackward0>)
tensor(4.)


In [50]:
#Option 3

x = torch.tensor(2.0,requires_grad=True)
print(x)

with torch.no_grad():
  y = x ** 2
  print(y)

#y.backward() will throw error


tensor(2., requires_grad=True)
tensor(4.)
